In [ ]:
import seaborn as sns
%matplotlib inline

import pandas as pd
import numpy as np
import sklearn.ensemble as ske

import tbtools.dev as tbdev
import tbtools.iter as tbiter
import tbtools.strings as tbstr

import utils.data as ud
import utils.plotting as up
import utils.evaluation.modelling.fit as mf

import os
import itertools
import pickle

# Find out how well models perform with greater lags

## Fetch data (if constructed)

In [ ]:
path = os.path.join(ud.paths.Paths.cache, 'rf_results2.pickle')
df = pd.read_pickle(path)

## Construct data

In [ ]:
def runforline(line, ra):
    for lag, dcwindow, boawindow, rn in \
    tbiter.IProgressBar(
        itertools.product(LAGS, TWINDOWS, TWINDOWS, RN),
        len(LAGS)*(len(TWINDOWS)**2)*len(RN)):

        with tbstr.silence():
            train, val, test = ud.design_matrices.get_by_settings(
                sample_step='2 min',
                split='all',
                line=line,
                lag=lag,
                dcwindow=dcwindow,
                rn=rn,
                boawindow=boawindow)

            features = [x for x in train if x!='y']
            data = {'train':train, 'val':val, 'test':test}
            ra.evaluation_sets = data

            rf = ske.RandomForestRegressor(n_estimators=k, n_jobs=-1)
            rf = rf.fit(train[features], train['y'])
            ra.append(('rf Line{line} L={lag} dcw={dcwindow} '
                       'boa={boawindow} rn={rn}').format(
                        line=line, lag=lag, dcwindow=dcwindow,
                        boawindow=boawindow, rn=rn), rf)

In [ ]:
def extract(field, df):
    return df.index.str.findall('{}=(\d+)'.format(field)).map(lambda x: x[0]).astype(int)

### 1st iteration

In [ ]:
LAGS = ('5 min', '10 min', '15 min', '20 min')
TWINDOWS = ('2 min', '3 min', '4 min')
RN = (5, 10, 15, 20)
k = 200

#### Line 1

In [ ]:
ra1 = mf.ResultsAggregator({}, target_name='y')
runforline(1, ra1)

In [ ]:
p1 = os.path.join(ud.paths.Paths.cache, 'rf_results_L1.pickle')

In [ ]:
# ra1.to_df().to_pickle(p1)

In [ ]:
df1 = pd.read_pickle(p1)

df1['lag'] = extract('L', df1)
df1['dcw'] = extract('dcw', df1)
df1['boa'] = extract('boa', df1)
df1['rn'] = extract('rn', df1)

df1 = df1.reset_index().drop('index', axis=1)

#### Line 2

In [ ]:
ra2 = mf.ResultsAggregator({}, target_name='y')
runforline(1, ra2)

In [ ]:
p2 = os.path.join(ud.paths.Paths.cache, 'rf_results_L2.pickle')

In [ ]:
ra2.to_df().to_pickle(p2)

In [ ]:
df2 = pd.read_pickle(p2)

df2['lag'] = extract('L', df2)
df2['dcw'] = extract('dcw', df2)
df2['boa'] = extract('boa', df2)
df2['rn'] = extract('rn', df2)

df2 = df2.reset_index().drop('index', axis=1)

#### Merge

In [ ]:
df = pd.concat([df1.assign(line=1), df2.assign(line=2)], ignore_index=True, axis=0)

In [ ]:
path = os.path.join(ud.paths.Paths.cache, 'rf_results.pickle')

In [ ]:
# df.to_pickle(path)

### 2nd iteration: More data! Now also for L=30 min

In [ ]:
LAGS = ('30 min', '60 min')
TWINDOWS = ('2 min', '3 min', '4 min')
RN = (5, 10, 15, 20)
k = 200

In [ ]:
ra_it2 = mf.ResultsAggregator({}, target_name='y')
runforline(1, ra_it2)
runforline(2, ra_it2)

In [ ]:
p_it2 = os.path.join(ud.paths.Paths.cache, 'rf_results_it2.pickle')

In [ ]:
ra_it2.to_df().to_pickle(p_it2)

In [ ]:
df_it2 = pd.read_pickle(p_it2)

df_it2['lag'] = extract('L', df_it2)
df_it2['dcw'] = extract('dcw', df_it2)
df_it2['boa'] = extract('boa', df_it2)
df_it2['rn'] = extract('rn', df_it2)
df_it2['line'] = df_it2.index.str.findall('Line(\d+)').map(lambda x: x[0]).astype(int)

df_it2 = df_it2.reset_index().drop('index', axis=1)

#### Merge and store on disk

In [ ]:
path_it1 = os.path.join(ud.paths.Paths.cache, 'rf_results.pickle')
df_it1 = pd.read_pickle(path_it1)

In [ ]:
df = pd.concat([df_it1, df_it2], ignore_index=True, axis=0)

In [ ]:
path_it2 = os.path.join(ud.paths.Paths.cache, 'rf_results2.pickle')

In [ ]:
df.to_pickle(path_it2)

## So how well do they perform?

Note: The following plot *is not* averages. The points are just not more distributed; `lag` explains almost all the variance. The effect of `bag-of-alarms window size` is minute. The effects of $R$ `window size` and $\Delta C$ `window size` are nonexistent.


In [ ]:
dfx = df.drop(['rn', 'dcw'], axis=1)

In [ ]:
import utils.plotting as up

In [ ]:
def plotres(res, title, save=False):
    for col, name in (('rmse','RMSE'), ('within_80%', 'RMSE 80th percentile')):
        dfy = pd.melt(res2[col].reset_index(), ['lag'], 
                      var_name='split', value_name=name)
        sns.factorplot(y=name, x='lag', hue='split', 
                       data=dfy, legend_out=True,
                       kind='point', size=6)
        if col=='rmse':
            sns.plt.ylim((0, 6.0))
        else:
            sns.plt.ylim((0, 8))
        if save:
            figname = 'lag_effect_{title}_{name}.png'.format(title=title, name=name.replace(' ', '_'))
            up.save_fig('analysis/results/'+figname)
            up.save_fig('w19/'+figname, target='week')

In [ ]:
model='RF'
res2 = dfx.drop(['boa', 'line'], axis=1).set_index('lag')
plotres(res2, model, save=True)

In [ ]:
# res = dfx.drop(['boa', 'line'], axis=1).set_index('lag')

# for col, name in (('rmse','RMSE'), ('within_80%', 'RMSE 80th percentile')):
#     dfy = pd.melt(res[col].reset_index(), ['lag'], 
#                   var_name='split', value_name=name)
#     sns.factorplot(y=name, x='lag', hue='split', 
#                    data=dfy, legend_out=True,
#                    kind='point', size=6)
#     figname = 'lag_effect_to60_{name}.png'.format(name=name.replace(' ', '_'))
#     up.save_fig('analysis/results/'+figname)
#     up.save_fig('w19/'+figname, target='week')

## Parameter importance analysis
rn and dcw changes are without consequence in the available data.

In [ ]:
path = os.path.join(ud.paths.Paths.cache, 'rf_results.pickle')
df = pd.read_pickle(path)

In [ ]:
import statsmodels.api as sm

#### Using OLS: We can see that OLS finds lag and boa to be most important.

In [ ]:
x = df[[
        'lag', 
#         'rn', 
        'boa', 
#         'dcw'
    ]].copy()
x = (x - x.mean()) / x.std()
x['intercept'] = 1
y = df[('rmse', 'val')]

ols = sm.OLS(y, x).fit()
ols.summary()

In [ ]:
sns.factorplot(y=('rmse','val'), x='lag', hue='boa', data=df[df.line==1])

#### OLS with interactions

It might seem that there is some interaction between boa and lag

In [ ]:
import statsmodels.formula.api as smf

In [ ]:
x = df[[
        'lag', 
        'rn', 
        'boa', 
        'dcw'
    ]].copy()

x = (x - x.mean()) / x.std()
x['intercept'] = 1

x['y'] = df[('rmse', 'val')]


formula = 'y ~ lag*rn*boa*dcw'
while True:
    ols = smf.ols(formula, x).fit()
    if ols.pvalues.max() > .05:
        c = ols.pvalues.argmax()
        formula += ' - ' + c
    else:
        print('Stopping with {} features left'.format(len(ols.pvalues)))
        break
        
# ols = smf.ols('y ~ lag*rn*boa*dcw', x).fit()
ols.summary()

In [ ]:
x = df[[
        'lag', 
        'rn', 
        'boa', 
        'dcw'
    ]].copy()

x = (x - x.mean()) / x.std()
x['intercept'] = 1

x['y'] = df[('rmse', 'train')]


formula = 'y ~ lag*rn*boa*dcw'
while True:
    ols = smf.ols(formula, x).fit()
    if ols.pvalues.max() > .05:
        c = ols.pvalues.argmax()
        formula += ' - ' + c
    else:
        print('Stopping with {} features left'.format(len(ols.pvalues)))
        break
        
# ols = smf.ols('y ~ lag*rn*boa*dcw', x).fit()
ols.summary()

#### Reshape dataframe to be used with sns.factorplot

Looong format

In [ ]:
df2 = df.drop('within_80%', axis=1)
df2.columns = [x[0] if x[0]!='rmse' else x[1] for x in df2.columns.ravel()]
df2 = pd.melt(df2, ['lag','dcw','boa','rn','line'], var_name='split', value_name='rmse')
df2 = pd.melt(df2, ['line', 'split', 'rmse'], var_name='feature', value_name='feature_value')
df2

### Plots of RMSE vs features

- Lag is dominating.
- validation has a slightly negative correlation with boa
- train has a slightly positive correlation with boa

Perhaps we can remove the effect of lag, and study what remains?

I did (see below), and found that boa also influences it, but the others do not.

#### rmse over feature for each split, for each feature, line 1

In [ ]:
# n=144
sns.factorplot(y='rmse',
               x='feature_value',
               row='split',
               col='feature',
               data=df2[df2.line==1], 
       palette='colorblind', kind='box');
up.save_fig('w19/rmse_feature_effect_l1.png', target='week')
up.save_fig('analysis/rmse_feature_effect_l1.png')

#### rmse over feature for each split, for each feature, line 2

In [ ]:
# n=144
sns.factorplot(y='rmse',
               x='feature_value',
               row='split',
               col='feature',
               data=df2[df2.line==2], 
       palette='colorblind', kind='box');
fig.tight_layout();

### Remove lag effect, see what happens then

Truly, the importance of BOA seems to be there. Mostly for the training set though. And in opposite directions for training and validation sets.

In [ ]:
rf = ske.RandomForestRegressor(100, n_jobs=-1)
rf = rf.fit(y=df['rmse'], X=df[['lag', 'line']])

In [ ]:
df3 = df[['rmse', 'lag', 'dcw', 'boa', 'rn', 'line']].copy()
df3['rmse'] = df3['rmse'] - rf.predict(df3[['lag', 'line']])

In [ ]:
df4 = df3.copy().drop('lag', axis=1)
df4.columns = [x[0] if x[0]!='rmse' else x[1] for x in df4.columns.ravel()]
df4 = pd.melt(df4, ['dcw','boa','rn','line'], var_name='split', value_name='rmse')
df4 = pd.melt(df4, ['line', 'split', 'rmse'], var_name='feature', value_name='feature_value')
# df4

#### Line 1

In [ ]:
# n=144
sns.factorplot(y='rmse',
               x='feature_value',
               row='split',
               col='feature',
               data=df4[df4.line==1], 
       palette='colorblind', kind='box');
# fig.tight_layout();
up.save_fig('w19/rmse_feature_effect_l1_rmlag.png', target='week')
up.save_fig('analysis/rmse_feature_effect_l1_rmlag.png')

#### Line 2

In [ ]:
# n=144
sns.factorplot(y='rmse',
               x='feature_value',
               row='split',
               col='feature',
               data=df4[df4.line==2], 
       palette='colorblind', kind='box');
fig.tight_layout();

### Remove boa effect, see what rn and dcw tell us

They tell us nothing at all.

In [ ]:
rf = ske.RandomForestRegressor(100, n_jobs=-1)
rf = rf.fit(y=df['rmse'], X=df[['boa', 'line']])

In [ ]:
df5 = df3[['rmse', 'dcw', 'boa', 'rn', 'line']].copy()
df5['rmse'] = df5['rmse'] - rf.predict(df5[['boa', 'line']])

In [ ]:
df6 = df5.copy().drop('boa', axis=1)
df6.columns = [x[0] if x[0]!='rmse' else x[1] for x in df6.columns.ravel()]
df6 = pd.melt(df6, ['dcw','rn','line'], var_name='split', value_name='rmse')
df6 = pd.melt(df6, ['line', 'split', 'rmse'], var_name='feature', value_name='feature_value')
# df4

#### Line 1

In [ ]:
# n=144
sns.factorplot(y='rmse',
               x='feature_value',
               row='split',
               col='feature',
               data=df6[df6.line==1], 
       palette='colorblind', kind='box');
fig.tight_layout();

#### Line 2

In [ ]:
# n=144
sns.factorplot(y='rmse',
               x='feature_value',
               row='split',
               col='feature',
               data=df6[df6.line==2], 
       palette='colorblind', kind='box');
fig.tight_layout();

# 

# Get some baselines
Predicting the mean, and predicting the last seen C value.

#### Setup

In [ ]:
def runforline_baseliner(line, ra, simplemodel, name):
    for lag, dcwindow, boawindow, rn in \
    tbiter.IProgressBar(
        itertools.product(LAGS, TWINDOWS, TWINDOWS, RN),
        len(LAGS)*(len(TWINDOWS)**2)*len(RN)):

        with tbstr.silence():
            train, val, test = ud.design_matrices.get_by_settings(
                sample_step='2 min',
                split='all',
                line=line,
                lag=lag,
                dcwindow=dcwindow,
                rn=rn,
                boawindow=boawindow)

            features = [x for x in train if x!='y']
            data = {'train':train, 'val':val, 'test':test}
            ra.evaluation_sets = data

            simplemodel = simplemodel.fit(train[features], train['y'])
            ra.append(('{name} Line{line} L={lag} dcw={dcwindow} '
                       'boa={boawindow} rn={rn}').format(name=name,
                        line=line, lag=lag, dcwindow=dcwindow,
                        boawindow=boawindow, rn=rn), simplemodel)

In [ ]:
LAGS = ('5 min', '10 min', '15 min', '20 min', '30 min', '60 min')
TWINDOWS = ('2 min', '3 min', '4 min')
RN = (5, 10, 15, 20)

In [ ]:
ra = mf.ResultsAggregator(evaluation_sets=None, target_name='y')

## Predicting the mean

In [ ]:
import sklearn.linear_model as skl

class MeanPredictor:

    def fit(self, x, y):
        self.lr = skl.LinearRegression(fit_intercept=False)
        self.lr = self.lr.fit(np.ones(y.shape).reshape(-1, 1), y)
        return self
    
    def predict(self, x):
        return self.lr.predict(np.ones(x.shape[0]).reshape(-1, 1))

In [ ]:
runforline_baseliner(1, ra, MeanPredictor(), 'MeanPredict')

In [ ]:
runforline_baseliner(2, ra, MeanPredictor(), 'MeanPredict')

In [ ]:
mp = ra.results['MeanPredict Line1 L=30 min dcw=3 min boa=2 min rn=15']['model']
mp.lr.coef_

## Predicting the last seen C value

In [ ]:
class LastCPredictor:

    def fit(self, x, y):
        return self
    
    def predict(self, x):
        cval = [v for v in x if v.startswith('C ')][0]
        return x[cval]

In [ ]:
runforline_baseliner(1, ra, LastCPredictor(), 'LastC')

In [ ]:
runforline_baseliner(2, ra, LastCPredictor(), 'LastC')

#### Merge

In [ ]:
res = ra.to_df()

res['lag'] = extract('L', res)
res['dcw'] = extract('dcw', res)
res['boa'] = extract('boa', res)
res['rn'] = extract('rn', res)
res['line'] = res.index.str.findall('Line(\d+)').map(lambda x: x[0]).astype(int)
res['model'] = res.index.str.findall('^(\w+)').map(lambda x: x[0]).astype(str)

res = res.reset_index().drop('index', axis=1)

## Display

In [ ]:
def plotres(res, title, save=False):
    for col, name in (('rmse','RMSE'), ('within_80%', 'RMSE 80th percentile')):
        dfy = pd.melt(res2[col].reset_index(), ['lag'], 
                      var_name='split', value_name=name)
        sns.factorplot(y=name, x='lag', hue='split', 
                       data=dfy, legend_out=True,
                       kind='point', size=6)
        if col=='rmse':
            sns.plt.ylim((0, 6.0))
        else:
            sns.plt.ylim((0, 8))
        if save:
            figname = 'lag_effect_{title}_{name}.png'.format(title=title, name=name.replace(' ', '_'))
            up.save_fig('analysis/results/'+figname)
            up.save_fig('w19/'+figname, target='week')

In [ ]:
model = 'LastC'
res2 = res[res.model == model].drop(['boa','line','rn','dcw','model'], axis=1).set_index('lag')
plotres(res2, model, save=True)

In [ ]:
# res = dfx.drop(['boa', 'line'], axis=1).set_index('lag')#.groupby(['lag']).mean()

model = 'MeanPredict'
res2 = res[res.model == model].drop(['boa','line','rn','dcw','model'], axis=1).set_index('lag')
plotres(res2, model, save=True)